## Datasets Levels

Weather radar datasets hosted on NCI consists of different product levels, namely:

- Level 1: Uncorrected radar data in the spherical coordinates. Typically used if you want to generate indepth products/analysis.

- Level 1b: Corrected and calibrated radar data in both spherical coordinates and Cartesian grids. Currently this is avalaible for CPOL, OceanPOL and CP2. The level 1b dataset is avalaible for NCI users on project rq0 and will be published externally later this year. Typically used if you want to skip the preprocessing steps. Corrections applied included those for attenuation, clutter removal and reflectivity calibration.

- Level 2: Derived products in 2D cartesian coodinates using level 1b radar data. These products are typically at a set altitude and include rainfall rate, classification, echo top height etc. Data is organised into daily netcdf files for rapid analysis using tools like xarrays.

## Datasets

_CPOL_

- CPOL radar was a polarimetric research weather radar in Darwin that operated from 1997-2017. The extra polarimetric information from this radar provides more robust retrievals of rainfall rate than the Z-R relationship.

- Level 1 data can be downloaded from the [ARM website](https://www.arm.gov/capabilities/instruments/cpol).

- Level 1b and 2 dataset hosted by NCI can be access through the [AURA](http://openradar.io) portal.

- The code repository used to generate the level 2 dataset from the level 1 archive is avaliable [here](https://github.com/vlouf/cpol_processing). Data licenced under CC4.0-BY-NC-SA. Contact Valentin if you are interested in publishing analysis.

_OceanPOL_

- We will be teaching using the level 1b OceanPOL radar dataset, which is hosted on NCI under project hj10. OceanPOL is a polarimetric weather radar that is part of the RV Investigator and has been operating duing voyages from 2017. Datasets are arranged by voyage. Data licenced under CC4.0-BY-NC-SA. Contact Valentin if you are interested in publishing analysis.

- Level 1b dataset hosted by NCI can be access through the [AURA](http://openradar.io) portal. The portal also includes a link to view voyage tracks.

- Dataset updated irreguarlly following voyages.

_CP2_

- CP2 was a polarmetric radar location near Brisbane that operated between 2007 and 2015 for research purposes. This unique radar collected both S and X band measurements, allowing for dual-frequency analysis. Data licenced under CC4.0-BY-NC-SA. Contact Joshua if you are interested in publishing analysis.

- Level 1 dataset hosted by NCI can be access through the [AURA](http://openradar.io) portal.

_Operational Network_

- The [Australian operation radar dataset](http://openradar.io/) is hosted on NCI under project rq0. This dataset contains 700+ years of weather radar data from the Australian network of 70+ radars and level 1 data updated daily. This dataset also includes newer polarimetric radars. Data licenced under CC4.0-BY-SA (commercial use permitted).

- The level 1 data from this project is published. Level 1b and level 2 data available for NCI users.

- To use the level 1 data, please reference the following [doi](http://dx.doi.org/10.25914/5cb686a8d9450).

- The code repository used to generate the level 1b and 2 dataset from the level 1 archive is avaliable [here](https://git.nci.org.au/jss548/open-radar).

_UQ-XPOL_

- Data from the mobile UQ-XPOL data will soon be avalaible on NCI (on request).

---

<img align="left" width="800" height="800" src="img/aura_webpage.png" border="5">



## Python (3.6+) Tools

Specific Radar Tools (for processing level 1 data)

- [PyART](https://arm-doe.github.io/pyart/): is a Python module containing a collection of weather radar algorithms and utilities. Py-ART is used by the Atmospheric Radiation Measurement (ARM) Climate Research Facility for working with data from a number of its precipitation and cloud radars, but has been designed so that it can be used by others in the radar and atmospheric communities to examine, processes, and analyze data from many types of weather radars.

- [wradlib](https://docs.wradlib.org/en/stable/): The ωradlib project has been initiated in order facilitate the use of weather radar data as well as to provide a common platform for research on new algorithms. ωradlib is designed to assist you in the most important steps of processing weather radar data. These may include: reading common data formats, georeferencing, converting reflectivity to rainfall intensity, identifying and correcting typical error sources (such as clutter or attenuation) and visualising the data.

Data Processing Tools

- [netCDF4](https://unidata.github.io/netcdf4-python/netCDF4/index.html): Python interface to the netCDF version 4 library (.nc files).
- [HDF5](https://www.hdfgroup.org/solutions/hdf5/): a Pythonic interface to the HDF5 binary data format (format used for Australian operational radars).
- [xarray](http://xarray.pydata.org/en/stable/): a package that makes working with labelled multi-dimensional arrays simple, efficient, and fun! allows for a more intuitive, more concise, and less error-prone experience. Builds upon the interfaces that pandas uses.
- [pandas](https://pandas.pydata.org/): like xarray but for tables (smart excel-like tools) 

Editor Tools

- [Jupyter Notebook](https://jupyter.org/): Provides an interactive browser-based environment for developing code
- [Pangeo Binder](https://binder.pangeo.io/): Turns a code repository into a collection of interactive Jupyter Notebooks without needing a Python environment on your local computer (used Google Compute Engine)

_How cool is that?_

<img align="left" width="300" src="https://pangeo-binder.readthedocs.io/en/prod/_images/binder_logo.png">



---

## Demonstration - Accessing Level 1 data from the operational network

(you don't need acccess to NCI for this data!)

In [ ]:
import os #used for system commands
import tempfile #used to create temporary folders to store data
import zipfile #used to extract tar files
import urllib #used to download data via http
from glob import glob #used for manipulating pathnames

from matplotlib import pyplot as plt #plotting toolkit
import pyart #python radar toolkit

"""
Options to define our data selection and plotting
"""

#Specific the radar and date we want to download
radar_id     = '15' #this is the Dampier radar in WA. if using a single digit, include a leading 0, e.g., '02'. Complete list of radar id's is on openradar.io
date_str     = '20200208' #in yyyymmdd
base_url     = 'http://dapds00.nci.org.au/thredds/fileServer/rq0' #base url for NCI server

#specify radar tilt and field to plot
tilt      = 2 #third tilt (python starts from 0)!
field     = 'DBZH' #reflectivity

In [ ]:
"""
here we build the server request url, download data and unzip
"""

#step 1: build request url and download zip file
zip_fn       = radar_id + '_' + date_str + '.pvol.zip'
request_url  = '/'.join([base_url, radar_id, date_str[0:4], 'vol', zip_fn])
print('my request is ',request_url)
#download the zip file
urllib.request.urlretrieve(request_url, zip_fn)

#step 2: extract the zip file to a temporary directory
temp_dir = tempfile.mkdtemp()
zip_fh = zipfile.ZipFile(zip_fn)
zip_fh.extractall(path = temp_dir)
zip_fh.close()

#step 3: list all the volumes extracted from the zip file
file_list = sorted(glob(temp_dir + '/*'))
#print out file list with index
for item in file_list:
    print(file_list.index(item),item)
#remove zip
os.system('rm ' + zip_fn)

In [ ]:
"""
select a file from the list and plot
"""

#select index from file list
my_index = 29
#read in radar object using pyart
radar = pyart.aux_io.read_odim_h5(file_list[my_index], file_field_names=True)
#generate display object
display = pyart.graph.RadarMapDisplay(radar)
#plot reflectivity field from a single tilt (this function includes a basemap)
fig = plt.figure(figsize=(16, 12), facecolor='w')
display.plot_ppi_map('DBZH', tilt,
                            vmin=-8, vmax=64, cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
                            resolution='10m')
#this may take a little time

# Non-meteorological returns (clutter)

The follow section introduces non-meteorological returns with weather radar (like an observations).
It is important to be aware of these issues when processing long time-series of data.

For some scientists these returns provide a unique dataset! e.g., [Meade et al. 2019](https://www.nature.com/articles/s41598-019-46549-2)

### Sources

Any object within the path of the transmitted beam will results in scattering. The magnitude and stability of the scattering relates to the target's size and movements. Non-meteorological targets are a hindrance for climate studies, but have many uses otherwise, including radar calibration and ecological monitoring.


- Ground
- Buildings
- Ships
- Aircraft
- Power lines
- Birds/bats
- Insects
- Ocean waves
- Suspended ash from bushfires or volcanoes
- The sun
- ~Unidentified Objects!?~ Debris from meteorites and rockets


<img align="center" width="800" src="img/IDR71_20191221.gif">

<img align="center" width="800" src="img/IDR63_20191225.gif">

### Removing clutter

Reflectivity (DBZH) products in the level 2 dataset have been processed to remove most ground clutter (some can still sneak through). It is difficult to remove smaller, non-stationary targets from convential single polarimetric data. Fortunately most clutter are stationary and can be removed by looking at the persistence of high reflectivity or using zero Doppler velocity.

See example below of unprocessed "DBZH" field and clean "DBZH_CLEAN" field from the level 1 archive. The "DBZH_CLEAN" field is not avalaible for older datasets. Polarimetic radars allow for more accurate clutter removal.

The new Rainbow, VIC weather radar (80km NW of Horsham)

<img align="left" width="800" height="800" src="img/IDR95_satellite.png" border="5">


<img align="left" width="800" height="800" src="img/IDR95_TH.png" border="5">

<img align="left" width="800" height="800" src="img/IDR95_DBZH.png" border="5">